## Select the dataset

In [15]:
dataset = "KIDNEY"

"KIDNEY"

# Load gene attributes

In [16]:
using CSV
using DataFrames
using Statistics

df = CSV.File("../$dataset/node_attributes.csv", typemap=Dict(Int => Float64)) |> DataFrame    # read the attribute file
df = unique(df, :name)      # remove possible duplicated genes
genenames = df[!, :name]
first(df, 15)

Row,name,label,label_wo_outliers,label_cluster2_kmeans5,label_cluster5_kmeans5,label_cluster4_kmeans5,gene_length,transcript_count,gc_content,official_gene_symbol,Gtex_kidney,gene_disease_ass_count,oncodb_expression,HPA_kidney_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole
,String,String3?,String3?,String3?,String3?,String3?,Float64?,Float64?,Float64?,String15?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,(clone tec14),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
2,100 kDa coactivator,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
3,14-3-3 tau splice variant,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
4,3-beta-hydroxysteroid dehydrogese,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
5,3-mercaptopyruvate sulfurtransferase variant,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
6,3'-phosphoadenosine-5'-phosphosulfate synthase,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
7,40S ribosomal protein S15a,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
8,5-aminoimidazole-4-carboxamide ribonucleotide formyltransferase,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
9,5'-3' exoribonuclease,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing


In [17]:
numeric_columns = []
for nm in names(df)
    if Float64 <: eltype(df[!, nm])
        append!(numeric_columns, [nm])
    end
end                                                                   # select only numeric attributes
select!(df, numeric_columns)
#df = df[typeintersect.(colwise(eltype, df), Number) .!= Union{}]      
for col in eachcol(df)                                                # fix missing and naN with mean value
    m = mean(collect(skipmissing(col)))
    std = mean(collect(skipmissing(col)))
    replace!(col, missing => m)
    replace!(col, NaN => m)
    convert.(Float64,col)
end
df = mapcols!(ByRow(Float64), df)                                     # convert Union{missing,Float64} to Float64 types

Row,gene_length,transcript_count,gc_content,Gtex_kidney,gene_disease_ass_count,oncodb_expression,HPA_kidney_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443
2,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443
3,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443
4,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443
5,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443
6,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443
7,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443
8,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443
9,70061.3,8.86662,46.6611,49.9335,8.87569,-0.240413,47.0997,4.55039,8.92982,5.09232,83.4247,4.56878,11.5978,69.4455,4.32581,249.813,0.848485,0.96286,0.5104,0.586593,0.921085,0.872414,0.725882,0.850397,0.475752,0.596558,0.666783,0.650443


### zscore attributes

In [18]:
using StatsBase
dt = fit(ZScoreTransform, Matrix(df), dims=2)
x = StatsBase.transform(dt, Matrix(df)) #|> DataFrame
df = DataFrame(x, names(df))
df[!, :name] = genenames

19296-element Vector{String}:
 "(clone tec14)"
 "100 kDa coactivator"
 "14-3-3 tau splice variant"
 "3-beta-hydroxysteroid dehydrogese"
 "3-mercaptopyruvate sulfurtransferase variant"
 "3'-phosphoadenosine-5'-phosphosulfate synthase"
 "40S ribosomal protein S15a"
 "5-aminoimidazole-4-carboxamide ribonucleotide formyltransferase"
 "5'-3' exoribonuclease"
 "6-O-methylguanine-D methyltransferase"
 "A0A087WZY1"
 "A0A0G2JLL6"
 "A0A0J9YVX5"
 ⋮
 "ZUP1"
 "ZW10"
 "ZWILCH"
 "ZWINT"
 "ZXDA"
 "ZXDB"
 "ZXDC"
 "ZYG11A"
 "ZYG11B"
 "ZYX"
 "ZZEF1"
 "ZZZ3"

## Load labels

In [19]:
dfl = CSV.File("../$dataset/node_labels.csv", typemap=Dict(Int => Float64)) |> DataFrame    # read the attribute file
dfl = unique(dfl, :name)         # remove gene duplicates
dfl = filter(:most_freq => !ismissing, dfl)

Row,name,ACH-000159,ACH-000189,ACH-000234,ACH-000246,ACH-000250,ACH-000262,ACH-000272,ACH-000313,ACH-000317,ACH-000375,ACH-000385,ACH-000411,ACH-000433,ACH-000484,ACH-000495,ACH-000600,ACH-000649,ACH-000684,ACH-000709,ACH-000792,ACH-000907,ACH-001163,ACH-001194,ACH-001310,ACH-001398,ACH-001532,ACH-001687,ACH-001688,ACH-002150,ACH-002189,most_freq
,String15,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3?,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3
1,A1BG,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE
2,A1CF,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,NE,NE,NE,aNE,NE,NE,aNE,aNE
3,A2M,NE,aNE,aNE,aNE,NE,NE,aNE,NE,NE,aNE,aNE,aNE,NE,aNE,NE,NE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE
4,A2ML1,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE
5,A3GALT2,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE
6,A4GALT,NE,NE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,aNE,NE,NE,NE,aNE,aNE,aNE,aNE
7,A4GNT,NE,aNE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE
8,AAAS,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aE,aNE,aNE,aNE,aNE,aNE
9,AACS,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE


## Select labels

In [20]:
dfl = filter(row -> row.most_freq ∈ ["E","NE"], dfl)

Row,name,ACH-000159,ACH-000189,ACH-000234,ACH-000246,ACH-000250,ACH-000262,ACH-000272,ACH-000313,ACH-000317,ACH-000375,ACH-000385,ACH-000411,ACH-000433,ACH-000484,ACH-000495,ACH-000600,ACH-000649,ACH-000684,ACH-000709,ACH-000792,ACH-000907,ACH-001163,ACH-001194,ACH-001310,ACH-001398,ACH-001532,ACH-001687,ACH-001688,ACH-002150,ACH-002189,most_freq
,String15,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3?,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3
1,A2ML1,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE
2,AADAC,NE,NE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,NE,NE,NE,NE,NE
3,AADACL3,NE,NE,NE,NE,NE,NE,aNE,aNE,NE,aNE,NE,NE,aNE,aNE,aNE,NE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,NE,NE,NE,NE,NE,aNE,NE
4,AADACL4,NE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE
5,AAGAB,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,aNE,NE,NE,NE,NE,aNE,NE,aNE,NE,NE,aNE,NE,NE,NE,aNE,aNE,NE,NE,NE,NE
6,AARS1,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E
7,AASS,aNE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,NE,NE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,NE,NE
8,ABCA10,aNE,NE,NE,NE,aNE,NE,NE,aNE,aNE,aNE,NE,NE,aNE,NE,NE,NE,aNE,NE,aNE,aNE,NE,NE,NE,aNE,aNE,NE,aNE,NE,aNE,aNE,NE
9,ABCA7,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE


In [21]:
genenames = dfl[!, :name]
dfall = innerjoin(dfl,df; on=:name)[!, Not(r"ACH-")]         # join labels and attributes dataframes

Row,name,most_freq,gene_length,transcript_count,gc_content,Gtex_kidney,gene_disease_ass_count,oncodb_expression,HPA_kidney_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole
,String15,String3,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,A2ML1,NE,5.1025,-0.18949,-0.186594,-0.190227,-0.189655,-0.19025,-0.19023,-0.189902,-0.190066,-0.189984,-0.180283,-0.189855,-0.189277,-0.187682,-0.189902,-0.177159,-0.190148,-0.190148,-0.19023,-0.190148,-0.190148,-0.190148,-0.190148,-0.190148,-0.19023,-0.190148,-0.190148,-0.190148
2,AADAC,NE,5.1021,-0.191903,-0.179672,-0.192628,-0.192271,-0.192726,-0.192564,-0.190067,-0.191536,-0.191903,-0.192271,-0.19096,-0.191536,-0.187863,-0.191536,-0.125053,-0.192271,-0.192271,-0.192638,-0.192271,-0.192271,-0.192271,-0.192271,-0.192271,-0.192271,-0.192271,-0.192271,-0.192271
3,AAGAB,NE,5.10242,-0.19044,-0.187288,-0.189219,-0.190731,-0.191158,-0.188431,-0.191023,-0.191023,-0.190828,-0.188013,-0.190676,-0.189994,-0.182965,-0.19044,-0.157627,-0.191023,-0.191023,-0.191023,-0.191023,-0.191023,-0.191023,-0.191023,-0.19112,-0.19112,-0.191023,-0.19112,-0.191023
4,AARS1,E,5.10231,-0.186921,-0.185857,-0.185807,-0.192535,-0.19285,-0.18664,-0.191553,-0.191693,-0.192114,-0.177657,-0.192676,-0.192255,-0.181868,-0.191974,-0.147338,-0.192697,-0.192681,-0.192744,-0.192734,-0.192687,-0.192694,-0.192714,-0.192697,-0.192749,-0.192732,-0.192722,-0.192725
5,AASS,NE,5.10247,-0.188652,-0.188019,-0.189985,-0.189515,-0.190521,-0.189515,-0.189847,-0.190112,-0.190046,-0.186926,-0.190311,-0.190245,-0.185664,-0.190378,-0.167872,-0.190378,-0.190378,-0.190444,-0.190444,-0.190378,-0.190378,-0.190378,-0.190378,-0.190378,-0.190444,-0.190444,-0.190444
6,ABCA10,NE,5.10252,-0.18866,-0.187407,-0.189302,-0.189257,-0.189378,-0.189327,-0.189148,-0.189257,-0.189257,-0.189094,-0.189311,-0.189203,-0.186438,-0.189148,-0.186492,-0.189311,-0.189311,-0.189365,-0.189365,-0.189311,-0.189311,-0.189311,-0.189311,-0.189311,-0.189311,-0.189311,-0.189311
7,ABCA7,NE,5.10233,-0.188907,-0.180102,-0.191129,-0.190564,-0.192581,-0.192406,-0.190978,-0.186837,-0.19015,-0.188493,-0.192427,-0.192013,-0.174622,-0.191599,-0.150399,-0.192427,-0.192427,-0.192427,-0.192427,-0.192427,-0.192427,-0.192427,-0.192427,-0.192427,-0.192427,-0.192427,-0.192427
8,ABCB7,E,5.10249,-0.189417,-0.188465,-0.189763,-0.189817,-0.190265,-0.189782,-0.189968,-0.189817,-0.190118,-0.183307,-0.190218,-0.190018,-0.18551,-0.190068,-0.173341,-0.190218,-0.190218,-0.190218,-0.190218,-0.190218,-0.190218,-0.190218,-0.190218,-0.190218,-0.190218,-0.190218,-0.190218
9,ABCC1,NE,5.10251,-0.18946,-0.188325,-0.189396,-0.189104,-0.189642,-0.18964,-0.189296,-0.189077,-0.189405,-0.185793,-0.189515,-0.189104,-0.187681,-0.189515,-0.182072,-0.189624,-0.189624,-0.189624,-0.189624,-0.189624,-0.189624,-0.189624,-0.189624,-0.189624,-0.189624,-0.189624,-0.189624


## Load embedding

In [22]:
dfe = CSV.File("../$dataset/embeddings/PPI_Node2Vec_64.csv") |> DataFrame    # read the attribute file
dfall = innerjoin(dfall,dfe; on=:name)               # join labels/attributes dataframe with embedding

Row,name,most_freq,gene_length,transcript_count,gc_content,Gtex_kidney,gene_disease_ass_count,oncodb_expression,HPA_kidney_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole,Node2Vec_1,Node2Vec_2,Node2Vec_3,Node2Vec_4,Node2Vec_5,Node2Vec_6,Node2Vec_7,Node2Vec_8,Node2Vec_9,Node2Vec_10,Node2Vec_11,Node2Vec_12,Node2Vec_13,Node2Vec_14,Node2Vec_15,Node2Vec_16,Node2Vec_17,Node2Vec_18,Node2Vec_19,Node2Vec_20,Node2Vec_21,Node2Vec_22,Node2Vec_23,Node2Vec_24,Node2Vec_25,Node2Vec_26,Node2Vec_27,Node2Vec_28,Node2Vec_29,Node2Vec_30,Node2Vec_31,Node2Vec_32,Node2Vec_33,Node2Vec_34,Node2Vec_35,Node2Vec_36,Node2Vec_37,Node2Vec_38,Node2Vec_39,Node2Vec_40,Node2Vec_41,Node2Vec_42,Node2Vec_43,Node2Vec_44,Node2Vec_45,Node2Vec_46,Node2Vec_47,Node2Vec_48,Node2Vec_49,Node2Vec_50,Node2Vec_51,Node2Vec_52,Node2Vec_53,Node2Vec_54,Node2Vec_55,Node2Vec_56,Node2Vec_57,Node2Vec_58,Node2Vec_59,Node2Vec_60,Node2Vec_61,Node2Vec_62,Node2Vec_63,Node2Vec_64
,String15,String3,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,FNTA,E,5.10242,-0.18985,-0.186917,-0.189737,-0.190376,-0.191252,-0.187551,-0.190055,-0.190465,-0.190671,-0.183283,-0.191184,-0.189439,-0.188824,-0.190568,-0.158043,-0.191184,-0.191184,-0.191184,-0.191184,-0.191184,-0.191184,-0.191184,-0.191184,-0.191184,-0.191184,-0.191184,-0.191184,0.0327755,0.84089,0.762798,-0.474988,0.375936,-0.453812,-0.882691,0.440906,-0.878124,-0.533896,-0.621145,-0.330792,-0.764699,0.298258,-0.163851,0.84922,1.10704,0.424903,0.638736,1.12484,0.359124,1.56491,-0.204588,-0.172318,0.181068,-0.754538,1.38996,1.04818,-0.605823,-0.513815,-0.21651,-0.83456,-0.530819,-1.30349,0.353133,0.521479,1.06585,-0.496735,0.561891,0.394384,-0.441063,0.400256,0.0466632,0.518116,0.640591,0.58238,0.449851,0.737192,-0.104968,-0.751855,-0.239735,0.285988,-0.0820393,-0.139406,0.196502,0.663146,0.158625,0.452507,-0.653603,-1.04916,1.30081,0.165904,0.00732472,-0.758322
2,ACVR1,NE,5.10251,-0.18906,-0.188515,-0.189722,-0.188985,-0.190045,-0.189038,-0.1894,-0.188153,-0.18974,-0.185395,-0.189891,-0.189604,-0.184564,-0.189967,-0.180332,-0.190005,-0.190005,-0.190005,-0.190005,-0.190005,-0.190005,-0.190005,-0.190005,-0.190043,-0.190005,-0.190005,-0.190005,-1.9554,0.210282,0.367882,-1.56708,-0.857939,0.62536,-0.870367,-0.319358,-1.63851,-0.667482,0.98611,-1.72713,-0.555335,-0.096736,0.910918,0.137879,0.822529,-0.558942,1.32688,0.258166,-0.912445,0.0193069,-0.232802,-0.604843,-0.581782,0.24931,-0.506253,0.69955,0.0970136,-1.24973,0.09171,-0.2101,0.27826,-0.191165,0.0811974,0.789027,0.682285,-0.969496,0.0484258,0.780362,-0.101702,-0.168606,0.425831,0.00610404,1.86621,1.21535,0.562664,0.0565809,0.312061,0.152257,-0.266875,-2.33547,-0.640375,0.371121,0.384757,0.496508,-1.29307,0.343799,0.0702841,-0.537535,-0.280513,1.06283,0.853472,-0.538201
3,PML,NE,5.10238,-0.190138,-0.187567,-0.191039,-0.190238,-0.192241,-0.191523,-0.191035,-0.185158,-0.190935,-0.157167,-0.191632,-0.189441,-0.184161,-0.191732,-0.171611,-0.19223,-0.19223,-0.19223,-0.19223,-0.19223,-0.19223,-0.19223,-0.19223,-0.19223,-0.19223,-0.19223,-0.19223,0.441455,0.529117,0.249046,-1.49291,0.443501,-0.0286639,-0.672491,0.785152,-0.116008,0.665382,0.48551

In [23]:
X = Matrix(dfall[!, Not(r"name|most_freq")])
y = vec(Matrix(DataFrames.select(dfall, [:most_freq])))
typeof(X), size(X), typeof(y), size(y)

(Matrix{Float64}, (5661, 92), Vector{String3}, (5661,))

## Classification with LightGBM

In [24]:
using MLJ
using MLJBase
using LightGBM
using StableRNGs
rng = StableRNG(566)
using CategoricalArrays

LIGHTGBM_SOURCE = abspath("~/LightGBM-3.2.0")
y = CategoricalArray(y)
rng = StableRNG(566)
model = LightGBM.MLJInterface.LGBMClassifier(
    objective = "multiclass", 
    num_iterations = 100,
    learning_rate = .1, 
    early_stopping_round = 15,
    num_leaves = 1000,
    #num_class = 2,
    #metric = ["matthews_correlation"]
)
evalmode = "cv"

preds = []
targets = []
if evalmode == "evaluate"
    cv=StratifiedCV(nfolds=5, shuffle=true, rng=rng)
    res = evaluate(model, X, y, resampling=cv, measure=[accuracy, balanced_accuracy, matthews_correlation])
    display(res)
elseif evalmode == "cv"
    stratified_cv = StratifiedCV(nfolds=5, rng=1234)
    rows = 1:size(X)[1]
    splits = MLJBase.train_test_pairs(stratified_cv, rows, y)
    for s in splits
        train, test = s[1], s[2]
        mach = machine(model, X, y, scitype_check_level=0)
        MLJ.fit!(mach, rows=train, verbosity=-1) 
        targets = cat(targets, vec(y[test, :]), dims=1)
        preds = cat(preds, MLJ.predict_mode(mach, rows=test), dims=1)
    end
    ŷ = convert(CategoricalArray{String3,1,UInt32},  preds)
    y_targets = convert(CategoricalArray{String3,1,UInt32}, targets)
    println("Accuracy: \t$(MLJBase.Accuracy()(ŷ,y_targets))")
    println("Balanced Acc: \t$(MLJBase.BalancedAccuracy()(ŷ,y_targets))")
    println("MCC: \t\t$(MLJBase.MatthewsCorrelation()(ŷ,y_targets))")
    confusionmatrix(ŷ,y_targets)
elseif evalmode == "split"
    train, test = partition(eachindex(y), 0.8, shuffle=true, rng=rng)
    mach = machine(model, X, y, scitype_check_level=0)
    MLJ.fit!(mach, rows=train, verbosity=-1) 
    ŷ = MLJ.predict_mode(mach, rows=test)
    y_targets = vec(y[test,:])
    println("Accuracy: \t$(MLJBase.Accuracy()(ŷ,y_targets))")
    println("Balanced Acc: \t$(MLJBase.BalancedAccuracy()(ŷ,y_targets))")
    println("MCC: \t\t$(MLJBase.MatthewsCorrelation()(ŷ,y_targets))")
    confusionmatrix(ŷ,y_targets)
end

   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`


Accuracy: 	0.9427662957074722
Balanced Acc: 	0.8435898348519364
MCC: 		0.7571640840693449


Row,NE,E,name
,Int64,Int64,String3
1,4752,77,NE
2,247,585,E


## Classification with RandomForest

In [25]:
using ScikitLearn.CrossValidation: cross_val_predict
using DecisionTree
using MLJBase
using MLJ
using CategoricalArrays

function confusionmatrix(predictions, labels)
   classes = vec(unique(labels))
   d = size(classes)[1]
   idx = Dict(zip(classes,Vector(1:d)))
   c = zeros(Int64, d,d)
   for i in 1:size(labels)[1]
       c[idx[labels[i]] ,idx[predictions[i]]] += 1 
   end 
   df = DataFrame(c, classes)
   df[!, :name] = classes
   return df
end
y_targets = convert(CategoricalArray{String3,1,UInt32}, y)
model = DecisionTree.RandomForestClassifier()
ŷ = convert(CategoricalArray{String3,1,UInt32}, cross_val_predict(model, X, y; cv=5))
println("Accuracy: \t$(MLJBase.Accuracy()(ŷ,y_targets))")
println("Balanced Acc: \t$(MLJBase.BalancedAccuracy()(ŷ,y_targets))")
println("MCC: \t\t$(MLJBase.MatthewsCorrelation()(ŷ,y_targets))")
confusionmatrix(ŷ,y)

Accuracy: 	0.9185656244479774
Balanced Acc: 	0.7940878526371126
MCC: 		0.6502135452353931


Row,E,NE,name
,Int64,Int64,String3
1,514,318,E
2,143,4686,NE
